In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import subprocess 
from brainsmash.workbench.geo import volume
import matplotlib.pyplot as plt
from brainsmash.mapgen.sampled import Sampled
import time
from tqdm import tqdm
import nibabel as nib
from glob import glob
import copy


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import cupy as cp
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
start = time.time()
mempool = cp.get_default_memory_pool()

# 1*17853

### Read in

In [3]:
data_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/FG_noBrainSmash/'
os.chdir(data_dir)


In [4]:
# The coordinates 
Cere_atlas = nib.load('result_cere2extra_gradient1__s_17853_nifti.nii')   
Cere = Cere_atlas.get_fdata()[Cere_atlas.get_fdata() != 0]
len(Cere) 
cere_coords = np.where(Cere_atlas.get_fdata() != 0)
cere_coords = [(cere_coords[0][x],
                cere_coords[1][x],
                cere_coords[2][x]) for x in range(len(cere_coords[0]))]
sample_coordinate = cere_coords

In [5]:
# Read in the FG
emb = np.load('embedding_dense_cere2extra_emb_z_17853.npy')
emb_temporary = emb[:,0:2]
emb_temporary = emb_temporary.T
sample_brainmap = emb[:,0:2]

## Pick the correct knn

In [35]:
# calculate distance matrix
from brainsmash.workbench.geo import volume
coord_file = np.array(sample_coordinate)
output_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/BrainSmash/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
filenames_cere = volume(coord_file, output_dir)


loading voxels coordinates from [[18 31 18]
 [18 31 19]
 [18 32 16]
 ...
 [72 33 16]
 [72 33 18]
 [72 33 20]]
file contains 17853 voxels
saving memory-mapped distance matrix files to /n02dat01/users/ypwang/Gradient/GeneticGradient/Data/BrainSmash/


In [36]:
output_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/BrainSmash/'
os.chdir(output_dir)

In [8]:
# # FG1: variogram fitting, 5000 seems the best
# from brainsmash.mapgen.eval import sampled_fit
# %matplotlib inline
# # These are three of the key parameters affecting the variogram fit, keep "knn" roughly consistent with the number of samples
# brain_map = sample_brainmap[:, 0]
# for i in range(1000,7000,500):
#     print(i)
#     kwargs = {'ns': 50,
#               'knn': i,
#               'pv': 70}
    
#     sampled_fit(brain_map, filenames_cere['D'], filenames_cere['index'], nsurr=10, **kwargs)
#     plt.savefig(f"SAfitting_plot_G1_{i}.jpg")

In [11]:
# # variogram fitting
# from brainsmash.mapgen.eval import sampled_fit
# %matplotlib inline
    
# # variogram fitting
# from brainsmash.mapgen.eval import sampled_fit
# %matplotlib inline
# # These are three of the key parameters affecting the variogram fit, keep "knn" roughly consistent with the number of samples
# knn = [5000, 1100]
# for i in range(sample_brainmap.shape[1]):
#     brain_map = sample_brainmap[:, i]
#     kwargs = {'ns': 50,
#                 'knn': knn[i],
#                 'pv': 70}
#     # Running this command will generate a matplotlib figure
#     sampled_fit(brain_map, filenames_cere['D'], filenames_cere['index'], nsurr=10, **kwargs)
#     # plt.savefig(output_dir + "/SAfitting_plot/" + str(i) + ".jpg")

## Generate the surrogate map

In [ ]:
# randomly generate surrogate brain maps with SA matched to target brain map
start = time.time()
permutation_times = 11000
ans = np.zeros([permutation_times, sample_brainmap.shape[0], sample_brainmap.shape[1]])
knn = [5000, 1100]
for i in tqdm(range(sample_brainmap.shape[1])):
    brain_map = sample_brainmap[:, 0]
    kwargs = {'ns': 50,
                'knn': knn[i],
                'pv': 70}
    gen = Sampled(x=brain_map, D=filenames_cere['D'], index=filenames_cere['index'], resample=False, n_jobs=50, **kwargs)
    surrogate_maps = gen(n = permutation_times)
    np.save(os.path.join(output_dir,'BrainSmash_resample_G' + str(i+0) +'_cere_17853_11000_noresample_cere2extra.csv'), surrogate_maps)
    ans[:, :, i] = surrogate_maps.reshape(permutation_times, sample_brainmap.shape[0])
np.save(os.path.join(output_dir,"BrainSmash_resample_G1+2_17853_11000_noresample_cere2extra.csv"), ans)
# np.save(os.path.join(output_dir,"BrainSmash_resample_G1+2_17853_11000_cere2extra.csv"), ans)

## Generate the nii

In [13]:
save_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/BrainSmash/Cere_G1+2_17853_cere2extra_multiprocessing/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
os.chdir(save_dir)

import multiprocessing
def add(x, y):
	return x+y

def merge_add(args):
	return add(*args)

In [28]:
ans = np.load(os.path.join(output_dir,"BrainSmash_resample_G1+2_17853_11000_resample_cere2extra.csv.npy"))
Grey_file = nib.load(os.path.join(data_dir,"HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii"))
axis2 = Grey_file.header.get_axis(1)
cerebellum_mask_ind = np.append(np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_LEFT'), 
                                np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT'))
cope1_file = nib.load(os.path.join(data_dir,"cope1.dtseries.nii"))

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [29]:
# memory limit to process respectively 
output = np.zeros([91282])
gg = 0
G = ans[:,:,gg]
def Value2nifti(i):
    print(i)
    output[cerebellum_mask_ind] = G[i,:]
    nib.cifti2.Cifti2Image(output.reshape([1,-1]), nib.cifti2.Cifti2Header(cope1_file.header.matrix)).to_filename(f'G{gg+1}_BrainSmash_{i+1}.dscalar.nii')
    cmd = f"wb_command -cifti-separate G{gg+1}_BrainSmash_{i+1}.dscalar.nii COLUMN -volume-all G{gg+1}_BrainSmash_{i+1}_nifti.nii"
    subprocess.check_output(cmd, shell=True);

In [16]:
# # Around 10 min
# output = np.zeros([91282])
# gg = 0
# G = ans[:,:,gg]
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)
# tasks = [[x] for x in sorted(range(len(G)))]
# for j in range(1000, 12000, 1000):
#     print(j)
#     print(f"\033[1;30;47m\t{j}\033[0m")
#     print(f'\033[0;36m {j} \033[0m')
#     pool.starmap(Value2nifti, tasks[j-1000:j])

In [30]:
output = np.zeros([91282])
gg = 1
G = ans[:,:,gg]
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
tasks = [[x] for x in sorted(range(len(G)))]
for j in range(1000, 12000, 1000):
    print(j)
    print(f"\033[1;30;47m\t{j}\033[0m")
    print(f'\033[0;36m {j} \033[0m')
    pool.starmap(Value2nifti, tasks[j-1000:j])

010413616840326480120481616011224728
128
56208
152144
88

96


200

192

176
224240
248184
216






232








1000
	1000
 1000 
21719725



57
41
161
241
73
11381

1456589


105153121137129




22549
1733233



201209
9

193
169249

185
177
26
2
4298

66
114
82
13058

138
15434

234
7450146

218

202242

1810226


210
170122

250
162
178
194
186
10690

27
3
4367

139
8399

171219

115
35
203
227
163
235243

251211
155

195
19187

59
147123

75
17911

51
131
91
107
28
444

68
100
140220
244

116
172
2043684228



236
196252

164
212
20
76
180
12415660

132

12188

5292

148
108
29
545

69
229
141
221
173101

245
85
165
205
117
2177

37
181253

19761213


125
237189
149
133

53
15713

30
10993

222
230
142
246
102174

166
182206

118
46622254



38
86
7854
62
70

214198

238134

190
14
150
126
94
158110

31
223
143
175
183
207
167
231
247103

215
119
25555199


4715187
135

191
39

7
2379

9571

63
111
15
239
256
264
272
159
127
280
288
296304

312
328320

344
336
352376360

384368




In [18]:
# # Read the 17853*1 into the nifti format, around 9h
# Grey_file = nib.load(os.path.join(data_dir,"HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii"))
# axis2 = Grey_file.header.get_axis(1)
# cerebellum_mask_ind = np.append(np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_LEFT'), 
#                                 np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT'))
# cope1_file = nib.load(os.path.join(data_dir,"cope1.dtseries.nii"))
# output = np.zeros([91282])
# for gg in range(len(ans.T)):
#     print(gg)
#     G = ans[:,:,gg]
#     for i in tqdm(range(len(G))):
#         output[cerebellum_mask_ind] = G[i,:]
#         img = nib.cifti2.Cifti2Image(output.reshape([1,-1]), nib.cifti2.Cifti2Header(cope1_file.header.matrix))
#         dscalar_name = f'G{gg+1}_BrainSmash_{i+1}.dscalar.nii'
#         nifti_name = f'G{gg+1}_BrainSmash_{i+1}_nifti.nii'
#         img.to_filename(dscalar_name)
#         cmd = "wb_command -cifti-separate " + str(dscalar_name) + " COLUMN -volume-all " +str(nifti_name)
#         subprocess.check_output(cmd, shell=True);
        
        

In [19]:
# # memory limit to process respectively 
# Grey_file = nib.load(os.path.join(data_dir,"HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii"))
# axis2 = Grey_file.header.get_axis(1)
# cerebellum_mask_ind = np.append(np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_LEFT'), 
#                                 np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT'))
# cope1_file = nib.load(os.path.join(data_dir,"cope1.dtseries.nii"))
# output = np.zeros([91282])
# gg = 0
# G = ans[:,:,gg]
# for i in tqdm(range(len(G))):
#     output[cerebellum_mask_ind] = G[i,:]
#     img = nib.cifti2.Cifti2Image(output.reshape([1,-1]), nib.cifti2.Cifti2Header(cope1_file.header.matrix))
#     dscalar_name = f'G{gg+1}_BrainSmash_{i+1}.dscalar.nii'
#     nifti_name = f'G{gg+1}_BrainSmash_{i+1}_nifti.nii'
#     img.to_filename(dscalar_name)
#     cmd = "wb_command -cifti-separate " + str(dscalar_name) + " COLUMN -volume-all " + str(nifti_name)
#     subprocess.check_output(cmd, shell=True);
      
        

In [23]:
Grey_file = nib.load(os.path.join(data_dir,"HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii"))
axis2 = Grey_file.header.get_axis(1)
cerebellum_mask_ind = np.append(np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_LEFT'), 
                                np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT'))
cope1_file = nib.load(os.path.join(data_dir,"cope1.dtseries.nii"))
output = np.zeros([91282])
gg = 1
G = ans[:,:,gg]
for i in tqdm(range(len(G))):
    output[cerebellum_mask_ind] = G[i,:]
    img = nib.cifti2.Cifti2Image(output.reshape([1,-1]), nib.cifti2.Cifti2Header(cope1_file.header.matrix))
    dscalar_name = f'G{gg+1}_BrainSmash_{i+1}.dscalar.nii'
    nifti_name = f'G{gg+1}_BrainSmash_{i+1}_nifti.nii'
    img.to_filename(dscalar_name)
    cmd = "wb_command -cifti-separate " + str(dscalar_name) + " COLUMN -volume-all " + str(nifti_name)
    subprocess.check_output(cmd, shell=True);
      

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
100%|██████████| 11000/11000 [5:08:14<00:00,  1.68s/it] 


## Get the FG

In [33]:
import multiprocessing
def disbtw(surf, aa):
    surf= np.array(surf)
    aa = np.expand_dims(aa,axis=0)
    dis = np.sqrt(((surf-aa)**2).sum(axis=1))
    return dis

def Grid2world(GridList, affine):
    """
    trans the grid coords into the corresponding MNI coords in MNI space
    GridList shape: N*3, affine shape 4*4
    """
    GridList = np.array(GridList)
    length = len(GridList)
    affine = np.array(affine)
    Grid_coord = np.concatenate([GridList, np.ones([length, 1])], axis=1)
    MNI_coord = Grid_coord.dot(affine.T)      
    MNI_coord = MNI_coord.round(2)
    return MNI_coord[:, :3]



def GetFG(sheet):
    y_gradient = np.zeros([len(loclist), 1])
    fid = int(sheet.split('/')[-1].split('_')[-2]) # get functional gradient id
    # print(fid)
    gradient_file = nib.load(sheet)
    gradient = gradient_file.get_fdata()
    affine = gradient_file.affine
    grid_list = np.nonzero(gradient)   
    grad_value = gradient[grid_list] 
    rrlist = []
    MNI_loc = Grid2world(np.array(grid_list).T, affine)
    for loc in loclist:
        dis = disbtw(MNI_loc, loc)
        tmp = grad_value[dis<4].mean()
        rrlist.append(tmp)
        # print(np.array(rrlist))
    y_gradient = rrlist
    return y_gradient

MNI= pd.read_csv('/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/report_4mm_317_cere2extra.csv')
# MNI= pd.read_csv('/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/report_337.csv')
loclist = MNI[['suit_x','suit_y','suit_z']].values.astype('float')

In [27]:
gg=0
nifti_name = f'G{gg+1}_BrainSmash_*_nifti.nii'
xpsheets = sorted(glob(nifti_name))
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
tasks = [[x] for x in sorted(glob(nifti_name))]
y_gradient_list = pool.starmap(GetFG, tasks)
y_gradient = pd.concat([pd.DataFrame(y_gradient_list[x]) for x in range(len(y_gradient_list))], axis=1)
filename = 'BrainSmash_Rad4mm_sample337_from17853_resample_G' + str(gg+1) + '.csv'
pd.DataFrame(y_gradient).to_csv(filename)

/tmp/ipykernel_32908/2660397743.py:36: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/n04dat04/ypwang/software/anaconda3/envs/gradient/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_32908/2660397743.py:36: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/tmp/ipykernel_32908/2660397743.py:36: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/n04dat04/ypwang/software/anaconda3/envs/gradient/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_32908/2660397743.py:36: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/tmp/ipykernel_32908/2660397743.py:36: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/tmp/ipykernel_32908/2660397743.py:36: RuntimeWar

In [34]:
gg=1
nifti_name = f'G{gg+1}_BrainSmash_*_nifti.nii'
xpsheets = sorted(glob(nifti_name))
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
tasks = [[x] for x in sorted(glob(nifti_name))]
y_gradient_list = pool.starmap(GetFG, tasks)
y_gradient = pd.concat([pd.DataFrame(y_gradient_list[x]) for x in range(len(y_gradient_list))], axis=1)
filename = 'BrainSmash_Rad4mm_sample317_from17853_resample_G' + str(gg+1) + '.csv'
pd.DataFrame(y_gradient).to_csv(filename)

/tmp/ipykernel_32908/3284511416.py:36: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/n04dat04/ypwang/software/anaconda3/envs/gradient/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
for gg in range(len(ans.T)):
    print(gg)
    nifti_name = f'G{gg+1}_BrainSmash_*_nifti.nii'
    xpsheets = sorted(glob(nifti_name))
    y_gradient = np.zeros([len(loclist), len(xpsheets)])
    for sheet in tqdm(xpsheets): 
        fid = int(sheet.split('/')[-1].split('_')[-2]) # get functional gradient id
        # print(fid)
        gradient_file = nib.load(sheet)
        gradient = gradient_file.get_fdata()
        affine = gradient_file.affine
        grid_list = np.nonzero(gradient)   
        grad_value = gradient[grid_list] 
        rrlist = []
        MNI_loc = Grid2world(np.array(grid_list).T, affine)
        for loc in loclist:
            dis = disbtw(MNI_loc, loc)
            tmp = grad_value[dis<4].mean()
            rrlist.append(tmp)
        # print(np.array(rrlist))
        y_gradient[:, fid-1] = rrlist
        filename = 'BrainSmash_Rad4mm_sample337_form17853_resample_G' + str(gg+1) + '.csv'
        pd.DataFrame(y_gradient).to_csv(filename)

0


  0%|          | 0/11000 [00:00<?, ?it/s]/tmp/ipykernel_65100/797664088.py:18: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/n04dat04/ypwang/software/anaconda3/envs/gradient/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 4/11000 [00:08<6:21:23,  2.08s/it]

In [22]:
gg=0
nifti_name = f'G{gg+1}_BrainSmash_*_nifti.nii'
xpsheets = sorted(glob(nifti_name))
y_gradient = np.zeros([len(loclist), len(xpsheets)])
for sheet in tqdm(xpsheets): 
    fid = int(sheet.split('/')[-1].split('_')[-2]) # get functional gradient id
    # print(fid)
    gradient_file = nib.load(sheet)
    gradient = gradient_file.get_fdata()
    affine = gradient_file.affine
    grid_list = np.nonzero(gradient)   
    grad_value = gradient[grid_list] 
    rrlist = []
    MNI_loc = Grid2world(np.array(grid_list).T, affine)
    for loc in loclist:
        dis = disbtw(MNI_loc, loc)
        tmp = grad_value[dis<4].mean()
        rrlist.append(tmp)
        # print(np.array(rrlist))
    y_gradient[:, fid-1] = rrlist
    filename = 'BrainSmash_Rad4mm_sample317_form17853_resample_G' + str(gg+1) + '.csv'
    pd.DataFrame(y_gradient).to_csv(filename)

  4%|▍         | 434/11000 [18:26<7:56:11,  2.70s/it]/tmp/ipykernel_32908/2573043607.py:17: RuntimeWarning: Mean of empty slice.
  tmp = grad_value[dis<4].mean()
/n04dat04/ypwang/software/anaconda3/envs/gradient/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 11000/11000 [13:09:04<00:00,  4.30s/it]  
